In [1]:
import tensorflow as tf
#tf.enable_eager_execution()

/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fmsnew/miniconda3/envs

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from nasbench.lib import cifar
from nasbench.lib import model_builder
from nasbench.lib import training_time
import numpy as np


from nasbench.lib import evaluate
from nasbench.lib import model_spec
from nasbench.lib import config as _config

from nasbench.scripts.run_evaluation import NumpyEncoder

import json

RESULTS_FILE = 'results.json'


In [3]:
_config

<module 'nasbench.lib.config' from '/home/fmsnew/misc/nasbench/nasbench/lib/config.py'>

In [4]:
import os
os.environ['AUTOGRAPH_VERBOSITY'] = '10'

In [5]:
config = _config.build_config()
config['use_tpu'] = False
config['train_data_files'] = ['train_1.tfrecords', 'train_2.tfrecords', 'train_3.tfrecords', 'train_4.tfrecords']
config['valid_data_file'] = ['validation.tfrecords']
config['test_data_file'] = ['test.tfrecords']
config['sample_data_file'] = ['sample.tfrecords']
config['train_epochs'] = 36

config['use_KD'] = True
config['imitation_lmb'] = 0.7
config['temperature'] = 20.0

for files_group in ['train_data_files', 'valid_data_file', 'test_data_file', 'sample_data_file']:
    for i in range(len(config[files_group])):
        config[files_group][i] = config[files_group][i].replace('.', '_KD.')

matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
      [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
      [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
      [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
      [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
      [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
      [0, 0, 0, 0, 0, 0, 0]]   # output layer
# Operations at the vertices of the module, matches order of matrix
labels=['input', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3', 'output']


matrix = np.array(matrix)
labels = np.array(labels)

spec = model_spec.ModelSpec(matrix, labels)
model_dir = '../my_model_dir_kd'

In [6]:
from nasbench.lib.evaluate import _TrainAndEvaluator

In [7]:
evaluator = _TrainAndEvaluator(spec, config, model_dir)

INFO:tensorflow:Using config: {'_model_dir': '../my_model_dir_kd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1073741824, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd686722518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=2, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=N

In [8]:
meta = evaluator.run()

Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done runni

INFO:tensorflow:vertex_channels: [256, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../my_model_dir_kd/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ../my_model_dir_kd/model.ckpt.
INFO:tensorflow:global_step/sec: 4.48072
INFO:tensorflow:examples/sec: 1147.06
INFO:tensorflow:global_step/sec: 5.51667
INFO:tensorflow:examples/sec: 1412.27
INFO:tensorflow:global_step/sec: 5.12329
INFO:tensorflow:examples/sec: 1311.56
INFO:tensorflow:global_step/sec: 5.12799
INFO:tensorflow:examples/sec: 1312.76
INFO:tensorflow:global_step/sec: 5.48333
INFO:tensorflow:examples/sec: 1403.73
INFO:tensorflow:global_step/sec: 5.1208
INFO

INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../my_model_dir_kd/model.ckpt-2812
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 

INFO:tensorflow:vertex_channels: [256, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-03-11:41:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../my_model_dir_kd/model.ckpt-5625
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [3/39]
INFO:tensorflow:Evaluation [6/39]
INFO:tensorflow:Evaluation [9/39]
INFO:tensorflow:Evaluation [12/39]
INFO:tensorflow:Evaluation [15/39]
INFO:tensorflow:Evaluation [18/39]
INFO:tensorflow:Evaluation [21/39]
INFO:tensorflow:Evaluation [24/39]
INFO:tensorflow:Evaluation [27/39]
INFO:tensorflow:Evaluation [30/39]
INFO:tensorflow:Evaluation [33/39]
INFO:tensorflow:Evaluation [36/39]
INFO:tensorflow:Evaluation [39/39]
INFO:tensorflow:Finished evaluation at 2020-02

In [9]:
[{k:v for k, v in res.items() if k != 'sample_metrics'} for res in meta['evaluation_results'] ]

[{'epochs': 0.0,
  'training_time': 0.0,
  'training_steps': 0,
  'train_accuracy': 0.110777244,
  'validation_accuracy': 0.10947516,
  'test_accuracy': 0.113080926,
  'predict_time': 4.282503128051758},
 {'epochs': 18.0,
  'training_time': 537.72,
  'training_steps': 2812,
  'train_accuracy': 0.83313304,
  'validation_accuracy': 0.7795473,
  'test_accuracy': 0.7723357,
  'predict_time': 4.270371675491333},
 {'epochs': 36.0,
  'training_time': 1081.959,
  'training_steps': 5625,
  'train_accuracy': 0.99869794,
  'validation_accuracy': 0.9055489,
  'test_accuracy': 0.90084136,
  'predict_time': 4.543731451034546}]

In [12]:
input_sample = cifar.CIFARInput('sample', config)

In [13]:
dataset = input_sample.input_fn({'batch_size':100, 'use_KD':config['use_KD']})

In [14]:
dataset

<PrefetchDataset shapes: ((100, 32, 32, 3), (100, 11)), types: (tf.float32, tf.float32)>

In [10]:
dataset = tf.data.TFRecordDataset(config['sample_data_file'])

In [15]:
for element in dataset: 
    #print(element)
    break

In [16]:
element

(<tf.Tensor: id=87, shape=(100, 32, 32, 3), dtype=float32, numpy=
 array([[[[ 1.7890141 ,  1.9495894 ,  2.1008997 ],
          [ 1.9477695 ,  2.0784345 ,  2.1158922 ],
          [ 2.027147  ,  2.0784345 ,  2.0859072 ],
          ...,
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ]],
 
         [[ 0.05858076,  0.27460143,  0.9314843 ],
          [ 0.51897126,  0.74166536,  1.2013494 ],
          [ 1.8525163 ,  1.965695  ,  2.0559223 ],
          ...,
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ]],
 
         [[-0.7986981 , -0.49846992,  0.4067466 ],
          [-0.5923162 , -0.2568851 ,  0.54167914],
          [ 0.8841086 ,  1.128201  ,  1.5611695 ],
          ...,
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.05

In [17]:
element[1]

<tf.Tensor: id=88, shape=(100, 11), dtype=float32, numpy=
array([[ 0.        , 27.231663  ,  2.5751529 , ..., -4.6600294 ,
         2.0751333 ,  2.8788328 ],
       [ 0.        , 16.118654  , -2.0107608 , ..., -7.676149  ,
         4.625721  , -3.3237    ],
       [ 0.        , 13.896395  , -3.0747104 , ..., -2.1238356 ,
        -7.08816   ,  4.364662  ],
       ...,
       [ 9.        ,  5.706644  ,  9.426671  , ..., -4.891599  ,
        -2.3245106 , 30.134348  ],
       [ 9.        , -1.8154393 ,  4.420699  , ..., -3.022294  ,
        -4.9302526 , 30.931654  ],
       [ 9.        ,  0.9914434 , 10.077514  , ..., -0.89257365,
         6.685395  , 15.760097  ]], dtype=float32)>

In [15]:
features = tf.parse_single_example(
      element,
      features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([11], tf.float32),
      })

In [16]:
features

{'image': <tf.Tensor: id=50, shape=(), dtype=string, numpy=b'\xee\xf8\xfd\xfd\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x81\x9e\xf2\xfe\xff\xfc\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffKX\xb5\xfd\xff\xfc\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffEEl\xe5\xfe\xfe\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffDAQ\x9c\xf8\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffH?Ob\xd0\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffQ@NS\x88\xf2\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff_APRU\xba\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x

In [20]:
from tensorflow.estimator import ModeKeys

In [21]:
features, labels, input_hooks = (
          evaluator.estimator._get_features_and_labels_from_input_fn(
input_sample.input_fn, ModeKeys.TRAIN))

KeyError: 'use_KD'

In [28]:
features

<tf.Tensor 'IteratorGetNext_1:0' shape=(256, 32, 32, 3) dtype=float32>

In [29]:
labels

<tf.Tensor 'IteratorGetNext_1:1' shape=(256,) dtype=int32>

In [30]:
config

{'train_data_files': ['train_1_KD.tfrecords',
  'train_2_KD.tfrecords',
  'train_3_KD.tfrecords',
  'train_4_KD.tfrecords'],
 'valid_data_file': ['validation_KD.tfrecords'],
 'test_data_file': ['test_KD.tfrecords'],
 'sample_data_file': ['sample_KD.tfrecords'],
 'data_format': 'channels_last',
 'num_labels': 10,
 'module_vertices': 7,
 'max_edges': 9,
 'available_ops': ['conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'],
 'stem_filter_size': 128,
 'num_stacks': 3,
 'num_modules_per_stack': 3,
 'batch_size': 256,
 'train_epochs': 36,
 'train_seconds': 14400.0,
 'learning_rate': 0.1,
 'lr_decay_method': 'COSINE_BY_STEP',
 'momentum': 0.9,
 'weight_decay': 0.0001,
 'max_attempts': 5,
 'intermediate_evaluations': ['0.5'],
 'num_repeats': 3,
 'use_tpu': False,
 'tpu_iterations_per_loop': 100,
 'tpu_num_shards': 2,
 'use_KD': True,
 'imitation_lmb': 0.7,
 'temperature': 20.0}